<img src='img/logo.png' alt='Drawing' style='width:2000px;'/>

# <font color=blue>4. 2D Frame Examples</font>
## <font color=blue>4.1. Elastic Frame Example</font>

In [ ]:
# import libraries 
#  ------------------------
import openseespy.opensees as ops # openseespy library
import matplotlib.pyplot as plt # a library to visualize data from Python
import openseespy.postprocessing.ops_vis as opsv # a visulization package for opensees
import openseespy.postprocessing.Get_Rendering as opsplt # another visulization package for opensees
import numpy as np # a library provides high-performance vector, matrix and higher-dimensional data structures for Python
%matplotlib inline

In [ ]:
# Define units
#  ------------------------

# Basic Units
m = 1.0
kN = 1.0
sec = 1.0

# Length
mm = m / 1000.0
cm = m / 100.0
inch = 25.4 * mm
ft = 12.0 * inch

# Force
N = kN / 1000.0
kips = kN * 4.448221615
lb = kips / 1.0e3

# Stress (kN/m2 or kPa)
Pa = N / (m ** 2)
kPa = Pa * 1.0e3
MPa = Pa * 1.0e6
GPa = Pa * 1.0e9
ksi = 6.8947573 * MPa
psi = 1e-3 * ksi

# Mass - Weight
tonne = kN * sec ** 2 / m
kg = N * sec ** 2 / m
lb = psi*inch**2

# Gravitational acceleration
g = 9.81*m/sec**2

# Time
min = 60*sec
hr = 60*min

In [ ]:
def generate_elastic_model():
    # clear memory of past model definitions
    ops.wipe('all')                                    
    
    # Create ModelBuilder (with two-dimensions and 3 DOF/node)
    ops.model('basic', '-ndm', 2, '-ndf', 3)

    # Create nodes
    # ------------

    # Set parameters for overall model geometry
    LBeam = 360.0*inch 
    LCol = 144.0*inch 
    HCol = 28*inch  # square-Column width
    BCol = HCol
    HBeam = 24*inch # Beam depth -- perpendicular to bending axis
    BBeam = 18*inch # Beam width -- parallel to bending axis

    # Create nodes
    ops.node(1, 0.0, 0.0)
    ops.node(2, LBeam, 0.0)
    ops.node(3, 0.0, LCol)
    ops.node(4, LBeam, LCol)
    ops.node(5, 0.0, 2*LCol)
    ops.node(6, LBeam, 2*LCol)
    
    # Fix supports at base of columns
    ops.fix(1, 1, 1, 1)
    ops.fix(2, 1, 1, 1)
    
    # Define material properties
    # ------------------------------------------
    fc = 4.0*ksi                    # Concrete strength
    Ec = 5000*MPa*(fc/MPa)**0.5     # Young's modulus
    Gc = Ec/(2*(1+0.2))             # Shear modulus

    # Define section properties
    # ------------------------------------------
    kv = 5/6                    # 5/6 for rectangular sections
    # columns
    Acol = HCol*BCol            # cross-sectional area of columns
    Icol = 1/12*BCol*HCol**3    # gross moment of inertia of columns
    Avcol = kv*Acol             # shear area for Timoshenko elements
    Abeam = BBeam*HBeam         # cross-sectional area of beams
    Ibeam = 1/12*BBeam*HBeam**3 # gross moment of inertia of beams
    Avbeam = kv*Abeam           # shear area for Timoshenko elements

    # Geometric transformations
    ColTransfTag = 1
    BeamTransfTag = 2
    ops.geomTransf('Linear', ColTransfTag)
    ops.geomTransf('Linear', BeamTransfTag)
    
    # Define column elements
    # ----------------------
    # ops.element('elasticBeamColumn', 11, 1, 3, Acol, Ec, Icol, ColTransfTag)
    # ops.element('elasticBeamColumn', 12, 2, 4, Acol, Ec, Icol, ColTransfTag)
    # ops.element('elasticBeamColumn', 21, 3, 5, Acol, Ec, Icol, ColTransfTag)
    # ops.element('elasticBeamColumn', 22, 4, 6, Acol, Ec, Icol, ColTransfTag)
    
    ops.element('ElasticTimoshenkoBeam', 11, 1, 3, Ec, Gc, Acol, Icol, Avcol, ColTransfTag)
    ops.element('ElasticTimoshenkoBeam', 12, 2, 4, Ec, Gc, Acol, Icol, Avcol, ColTransfTag)
    ops.element('ElasticTimoshenkoBeam', 21, 3, 5, Ec, Gc, Acol, Icol, Avcol, ColTransfTag)
    ops.element('ElasticTimoshenkoBeam', 22, 4, 6, Ec, Gc, Acol, Icol, Avcol, ColTransfTag)
    
    # Define beam elments
    # -----------------------------
    # ops.element('elasticBeamColumn', 13, 3, 4, Abeam, Ec, Ibeam, BeamTransfTag)
    # ops.element('elasticBeamColumn', 23, 5, 6, Abeam, Ec, Ibeam, BeamTransfTag)

    ops.element('ElasticTimoshenkoBeam', 13, 3, 4, Ec, Gc, Abeam, Ibeam, Avbeam, BeamTransfTag)
    ops.element('ElasticTimoshenkoBeam', 23, 5, 6, Ec, Gc, Abeam, Ibeam, Avbeam, BeamTransfTag)

    # Define gravity loads
    # --------------------
    P = 0.1*fc*HCol*BCol  # 10% of axial capacity of columns

    # Create a Plain load pattern with a Linear TimeSeries
    ops.timeSeries('Linear', 1)
    ops.pattern('Plain', 1, 1)

    # Create nodal loads at nodes
    ops.load(3, 0.0, -P, 0.0)
    ops.load(4, 0.0, -P, 0.0)
    ops.load(5, 0.0, -P, 0.0)
    ops.load(6, 0.0, -P, 0.0) 
    
    # Alternatively, apply distributed loading along the beam
#     w = (2*P)/LBeam
#     ops.eleLoad('-ele', 13, '-type', '-beamUniform', -w, 0)
#     ops.eleLoad('-ele', 23, '-type', '-beamUniform', -w, 0)
    
    # Define nodal mass in terms of axial load on columns
    mass = P/g

    ops.mass(3, mass, mass, 0.0)
    ops.mass(4, mass, mass, 0.0)
    ops.mass(5, mass, mass, 0.0)
    ops.mass(6, mass, mass, 0.0)

In [ ]:
# Display the Numerical Model for Elastic Frame
generate_elastic_model()

# plot model with tag lebels
opsplt.plot_model('nodes','elements')
opsv.plot_model()

## <font color=blue>4.1.1. Gravity Analysis</font>

In [ ]:
def do_gravity(pflag = 0):
    ops.wipeAnalysis() # Remove previous analysis objects
    
    # Gravity-analysis: load-controlled static analysis
    Tol = 1e-8 # convergence tolerance for test
    NstepGravity = 500
    DGravity = 1/NstepGravity
    ops.integrator('LoadControl', DGravity) # determine the next time step for an analysis
    ops.numberer('Plain') # renumber dof's to minimize band-width (optimization), if you want to
    ops.system('BandGeneral') # how to store and solve the system of equations in the analysis
    ops.constraints('Transformation') # how it handles boundary conditions
    ops.test('NormDispIncr', Tol, 50) # determine if convergence has been achieved at the end of an iteration step
    ops.algorithm('Newton') # use Newton's solution algorithm: updates tangent stiffness at every iteration
    ops.analysis('Static') # define type of analysis static or transient
    ops.analyze(NstepGravity) # apply gravity

    # maintain constant gravity loads and reset time to zero
    ops.loadConst('-time', 0.0)
    
    # Get the reaction forces in vertical direction, this is for verification purposes
    ops.reactions('-dynamic', '-rayleigh')
    
    if pflag == 1:
        F_tot=0
        for node in ops.getNodeTags():
            forces = np.array(ops.nodeReaction(node))
            F_tot += forces
            print('Nodal reaction for node %d: Fx=%.3f, Fy=%.3f, M=%.3f' %(node,forces[0],forces[1],forces[2]))
        print('Total reaction: Fx=%.3f, Fy=%.3f, M=%.3f' %(F_tot[0],F_tot[1],F_tot[2]))
    
    # print ele forces
    eleForces = {}
    for ele in ops.getEleTags():
        forces = ops.eleForce(ele)
        eleForces[ele] = np.array(forces)
    
    nodalDisps = {}
    for node in ops.getNodeTags():
        disps = ops.nodeDisp(node)
        nodalDisps[node] = np.array(disps)
        
    return eleForces, nodalDisps

In [ ]:
# Static Analysis of the elastic frame under gravity loads
# generate_elastic_model() # generate a clean model
gravityForces, gravityDisps = do_gravity(pflag = 1)

## <font color=blue>4.1.2. Modal Analysis</font>

In [ ]:
# Modal Analyses of the elastic frame
# In the past version of OpenSees there was no particular function to calculate modal properties
# Hence ,I was using my own function which you can find it here:
# https://github.com/volkanozsarac/ModalAnalysis-OpenSeespy.git
# Now there is a brand new function developed by Dr. Massimo Petracca
# documentation is here: 
# https://opensees.github.io/OpenSeesDocumentation/user/manual/analysis/modalProperties.html

generate_elastic_model() # generate a clean model
nModes = 2 # desired number of modes to calculate 
lambdas = ops.eigen(nModes) # eigenvalue analysis for nEigen modes
ops.modalProperties("-print", "-file", "ModalReport.txt", "-unorm")

In [ ]:
# Display specific mode shape with scale factor of 20 using the active model
opsplt.plot_modeshape(1, 20)
opsplt.plot_modeshape(2, 20)

## <font color=blue>4.1.3. Response Spectrum Analysis</font>

In [ ]:
def do_rsa(nModes, method = 'CQC', xi = 0.05, aDir = 1):
    # nModes: number of modes to be used for analysis
    # method: modal combination method 'CQC' or 'SRSS'
    # xi: damping for CQC modal combination
    # aDir: analysis direction
    
    # we need a function to perform CQC combination of modes.
    # CQC function
    def CQC(mu, lambdas, dmp):
        # mu: results of a node or an element for each mode (forces, displacements, deformations etc.)
        # lambdas: eigenvalues
        # dmp: damping ratio for modes
        u = 0.0
        ne = len(lambdas)
        for i in range(ne):
            for j in range(ne):
                di = dmp[i]
                dj = dmp[j]
                bij = lambdas[i]/lambdas[j]
                rho = ((8.0*((di*dj)**0.5)*(di+bij*dj)*(bij**(3.0/2.0))) /
                    ((1.0-bij**2.0)**2.0 + 4.0*di*dj*bij*(1.0+bij**2.0) + 
                    4.0*(di**2.0 + dj**2.0)*bij**2.0))
                u += mu[i] * mu[j] * rho
        return (u)**0.5

    def SRSS(mu, nModes):
        # mu: results of a node or an element for each mode (forces, displacements, deformations etc.)
        u = 0.0
        for i in range(nModes):
            u += mu[i]**2
        return (u)**0.5
    
    # Import spectrum
    data = np.loadtxt('tools//Target_Spectrum.txt')
    Periods = data[:,0]
    Sa = data[:,1]
    
    # some settings for the response spectrum analysis
    tsTag = 2 # use the timeSeries 1 as response spectrum function
    direc = 1 # excited DOF = Ux
    ops.timeSeries("Path",tsTag,"-time",*Periods,"-values",*Sa,"-factor",9.806)
    
    # define new analysis settings
    ops.constraints("Transformation")
    ops.numberer("RCM")
    ops.system("UmfPack")
    ops.test("NormUnbalance", 0.0001, 10)
    ops.algorithm("Linear")
    ops.integrator("LoadControl", 0.0)
    ops.analysis("Static")
    
    # first compute the modal properties, and noramlize modes.
    nEigen = 2                                        # desired number of modes to calculate 
    lambdas = ops.eigen('-genBandArpack', nEigen)    # eigenvalue analysis for nEigen modes
    ops.modalProperties("-unorm")
    # currently we use the same damping for each mode (5%)
    dmp = [xi]*nEigen
    
    # ========================================================================
    # run a response spectrum analysis mode-by-mode.
    # grab results during the loop
    # then do modal combination in post-processing.
    # ========================================================================
    ModalForces = {}
    for ele in ops.getEleTags():
        ModalForces[ele] = []
        
    ModalDisps = {}
    for node in ops.getNodeTags():
        ModalDisps[node] = []
        
    for i in range(len(lambdas)):
        ops.responseSpectrum(tsTag, aDir, '-mode', i+1)
        for ele in ops.getEleTags():
            forces = np.array(ops.eleResponse(ele, 'force'))
            ModalForces[ele].append(forces)
        for node in ops.getNodeTags():
            disps = np.array(ops.nodeDisp(node))
            ModalDisps[node].append(disps)
    
    # post process the results doing the CQC modal combination for element forces and nodal displacements
    eleForces = {}
    for ele in ops.getEleTags():
        if method == 'CQC':
            forces = CQC(ModalForces[ele], lambdas, dmp)
        elif method == 'SRSS':
            forces = SRSS(ModalForces[ele], nModes)
        eleForces[ele] = forces
    
    nodalDisps = {}
    for node in ops.getNodeTags():
        if method == 'CQC':
            disps = CQC(ModalDisps[node], lambdas, dmp)
        elif method == 'SRSS':
            disps = SRSS(ModalDisps[node], nModes)

        nodalDisps[node] = disps
    
    return eleForces, nodalDisps

In [ ]:
# Response Spectrum Analysis of the elastic frame  
generate_elastic_model() # generate a clean model
rsaForces, rsaDisps = do_rsa(nModes, 'SRSS')
for ele in rsaForces.keys():
    print('Element ID: ', ele)
    print('i end:', 'F1=',rsaForces[ele][0], 'F2=',rsaForces[ele][1], 'M=',rsaForces[ele][2])
    print('j end:', 'F1=',rsaForces[ele][3], 'F2=',rsaForces[ele][4], 'M=',rsaForces[ele][5])

## <font color=blue>4.2. Inelastic Frame Example</font>

In [ ]:
def BuildRCrectSection (SecID, HSec, BSec, coverH, coverB, coreID, coverID, steelID, 
                        numBarsTop, barAreaTop, numBarsBot, barAreaBot, numBarsIntTot, barAreaInt, 
                        nfCoreY=20, nfCoreZ=20, nfCoverY=20, nfCoverZ=20):
    """
     Build fiber rectangular RC section, 1 steel layer top, 1 bot, 1 skin, confined core
     Define a procedure which generates a rectangular reinforced concrete section
     with one layer of steel at the top & bottom, skin reinforcement and a 
     confined core.
            by: Volkan Ozsarac, 2021
                adapted from Silvia Mazzoni, 2006
     
     Formal arguments
        SecID - tag for the section that is generated by this procedure
        HSec - depth of section, along local-y axis
        BSec - width of section, along local-z axis
        cH - distance from section boundary to neutral axis of reinforcement
        cB - distance from section boundary to side of reinforcement
        coreID - material tag for the core patch
        coverID - material tag for the cover patches
        steelID - material tag for the reinforcing steel
        numBarsTop - number of reinforcing bars in the top layer
        numBarsBot - number of reinforcing bars in the bottom layer
        numBarsIntTot - TOTAL number of reinforcing bars on the intermediate layers, symmetric about z axis and 2 bars per layer-- needs to be an even integer
        barAreaTop - cross-sectional area of each reinforcing bar in top layer
        barAreaBot - cross-sectional area of each reinforcing bar in bottom layer
        barAreaInt - cross-sectional area of each reinforcing bar in intermediate layer 
        nfCoreY - number of fibers in the core patch in the y direction
        nfCoreZ - number of fibers in the core patch in the z direction
        nfCoverY - number of fibers in the cover patches with long sides in the y direction
        nfCoverZ - number of fibers in the cover patches with long sides in the z direction
        
                            y
                            ^
                            |     
                 ----------------------- --        |
                 |   o      o      o   | -- coverH |
                 |                     |           |
                 |   o             o   |           |
          z <--- |          +          |       HSec|
                 |   o             o   |           |
                 |                     |           |
                 |   o o o o o o o o   | --coverH  |
                 ----------------------- --        |
                 |--------Bsec---------|
                 |---|   coverB   |----|
    
                           y
                           ^
                           |    
                 ----------------------
                 |\      cover       /|
                 | \------Top-------/ |
                 |c|                |c|
                 |o|                |o|
         z <-----|v|      core      |v|  HSec
                 |e|                |e|
                 |r|                |r|
                 | /-------Bot------\ |
                 |/       cover      \|
                 ----------------------
                           Bsec
        
    
     Notes
        The core concrete ends at the NA of the reinforcement
        The center of the section is at (0,0) in the local axis system
    """
    coverY = HSec/2.0            # The distance from the section z-axis to the edge of the cover concrete -- outer edge of cover concrete
    coverZ = BSec/2.0            # The distance from the section y-axis to the edge of the cover concrete -- outer edge of cover concrete
    coreY = coverY-coverH        # The distance from the section z-axis to the edge of the core concrete --  edge of the core concrete/inner edge of cover concrete
    coreZ = coverZ-coverB        # The distance from the section y-axis to the edge of the core concrete --  edge of the core concrete/inner edge of cover concrete
    numBarsInt = numBarsIntTot/2 # number of intermediate bars per side

    # Define the fiber section
    ops.section('Fiber', SecID)
    # Define the core patch
    ops.patch('quad', coreID, nfCoreZ, nfCoreY, -coreY, coreZ, -coreY, -coreZ, coreY, -coreZ, coreY, coreZ)

    # Define the four cover patches
    ops.patch('quad', coverID, 2, nfCoverY, -coverY, coverZ, -coreY,   coreZ,   coreY,   coreZ,  coverY, coverZ)
    ops.patch('quad', coverID, 2, nfCoverY, -coreY, -coreZ,  -coverY, -coverZ,  coverY, -coverZ, coreY,  -coreZ)
    ops.patch('quad', coverID, nfCoverZ, 2, -coverY, coverZ, -coverY, -coverZ, -coreY,  -coreZ, -coreY,   coreZ)
    ops.patch('quad', coverID, nfCoverZ, 2,  coreY,  coreZ,   coreY,  -coreZ,   coverY, -coverZ, coverY, coverZ)    

    # define reinforcing layers
    ops.layer('straight', steelID ,numBarsInt ,barAreaInt, -coreY,  coreZ,  coreY,  coreZ) # intermediate skin reinf. +z
    ops.layer('straight', steelID ,numBarsInt ,barAreaInt, -coreY, -coreZ,  coreY, -coreZ) # intermediate skin reinf. -z
    ops.layer('straight', steelID ,numBarsTop ,barAreaTop,  coreY,  coreZ,  coreY, -coreZ) # top layer reinfocement
    ops.layer('straight', steelID ,numBarsBot ,barAreaBot, -coreY,  coreZ, -coreY, -coreZ) # bottom layer reinforcement


In [ ]:
def generate_inelastic_model():

    # clear memory of past model definitions
    ops.wipe('all')                                    
    
    # Create ModelBuilder (with two-dimensions and 3 DOF/node)
    ops.model('basic', '-ndm', 2, '-ndf', 3)

    # Set parameters for overall model geometry
    LBeam = 360.0*inch 
    LCol = 144.0*inch 
    HCol = 28*inch  # square-Column width
    BCol = HCol
    HBeam = 24*inch # Beam depth -- perpendicular to bending axis
    BBeam = 18*inch # Beam width -- parallel to bending axis

    # Create nodes
    ops.node(1, 0.0, 0.0)
    ops.node(2, LBeam, 0.0)
    ops.node(3, 0.0, LCol)
    ops.node(4, LBeam, LCol)
    ops.node(5, 0.0, 2*LCol)
    ops.node(6, LBeam, 2*LCol)
    
    # Fix supports at base of columns
    ops.fix(1, 1, 1, 1)
    ops.fix(2, 1, 1, 1)

    # Geometric transformations
    ColTransfTag = 1
    BeamTransfTag = 2
    ops.geomTransf('PDelta', ColTransfTag)  # includes P-Delta effects
    ops.geomTransf('Linear', BeamTransfTag) # does not include P-Delta effects

    # Define materials for nonlinear columns
    # ------------------------------------------
    # CONCRETE
    fc = 4.0*ksi   # nominal concrete compressive strength at 28 days
    Ec = 5000*MPa*(fc/MPa)**0.5  # Young's modulus
    nu = 0.2 # poisson's ratio
    Gc = Ec/(2*(1+nu)) # Shear modulus
    # unconfined concrete
    IDconcCover = 1
    Kres = 0.2          # ratio of residual/ultimate to maximum stress
    fc1U = -fc          # UNCONFINED concrete (todeschini parabolic model), maximum stress
    eps1U = -0.003      # strain at maximum strength of unconfined concrete
    fc2U = Kres*fc1U    # ultimate stress
    eps2U = -0.01       # strain at ultimate stress
    # confined concrete
    IDconcCore = 2
    Kfc = 1.3           # ratio of confined to unconfined concrete strength
    fc1C = Kfc*fc1U     # CONFINED concrete (mander model), maximum stress
    eps1C = 2*fc1C/Ec   # strain at maximum stres
    fc2C = Kres*fc1C    # ultimate stress
    eps2C = 20*eps1C    # strain at ultimate stress 
    # tensile-strength properties
    Lambda = 0.1        # ratio between unloading slope at eps2 and initial slope Ec
    fct = 0.14*fc       # tensile strength +tension
    Ets = fct/0.002     # tension softening stiffness
    ops.uniaxialMaterial('Concrete02', IDconcCover, fc1U, eps1U, fc2U, eps2U, Lambda, fct, Ets)
    ops.uniaxialMaterial('Concrete02', IDconcCore,  fc1C, eps1C, fc2C, eps2C, Lambda, fct, Ets)

    # STEEL
    # Reinforcing steel
    fsy = 66.8*ksi;  # Yield stress
    Es = 29000*ksi;  # Young's modulus
    bs = 0.01 # strain-hardening ratio
    IDSteel = 3
    ops.uniaxialMaterial('Steel01', IDSteel, fsy, Es, bs)
    IDMinMaxSteel = 4
    minStrain = -0.1  # minimum steel strain in the fibers (steel buckling)
    maxStrain = 0.1  # maximum steel strain in the fibers (steel rupture)
    ops.uniaxialMaterial('MinMax', IDMinMaxSteel, IDSteel, '-min', minStrain, '-max', maxStrain)

    # Shear material for columns
    kv = 5/6 # shear area factor timoshenko beam
    Av = HCol*BCol*kv # shear area
    shear_mat = 5
    ops.uniaxialMaterial('Elastic', shear_mat, Gc*Av)
    
    # Define Cross-Sections
    # ------------------------------------------
    # rectangular section with one layer of steel evenly distributed around the perimeter and a confined core.
    ColSecTag = 1
    BeamSecTag = 2
    cover = 2.5*inch  # rectangular-RC-Column cover
    numBarsTopCol = 8 # number of longitudinal-reinforcement bars on top layer
    numBarsBotCol = 8 # number of longitudinal-reinforcement bars on bottom layer
    numBarsIntCol = 6 # TOTAL number of reinforcing bars on the intermediate layers
    barAreaTopCol = 1*inch**2 # longitudinal-reinforcement bar area
    barAreaBotCol = 1*inch**2 # longitudinal-reinforcement bar area
    barAreaIntCol = 1*inch**2 # longitudinal-reinforcement bar area

    numBarsTopBeam = 6 # number of longitudinal-reinforcement bars on top layer
    numBarsBotBeam = 6 # number of longitudinal-reinforcement bars on bottom layer
    numBarsIntBeam = 2 # TOTAL number of reinforcing bars on the intermediate layers
    barAreaTopBeam = 1*inch**2 # longitudinal-reinforcement bar area
    barAreaBotBeam = 1*inch**2 # longitudinal-reinforcement bar area
    barAreaIntBeam = 1*inch**2 # longitudinal-reinforcement bar area
    
    BuildRCrectSection(ColSecTag, HCol, BCol, cover, cover, IDconcCore, IDconcCover, IDMinMaxSteel, 
                      numBarsTopCol, barAreaTopCol, numBarsBotCol, barAreaBotCol, numBarsIntCol, barAreaIntCol) 
    BuildRCrectSection(BeamSecTag, HBeam, BBeam, cover, cover, IDconcCore, IDconcCover, IDMinMaxSteel, 
                      numBarsTopBeam, barAreaTopBeam, numBarsBotBeam, barAreaBotBeam, numBarsIntBeam, barAreaIntBeam)
    
    # Aggregate shear stiffness for columns to account for shear deformations
    ops.section('Aggregator', 4, shear_mat, 'Vy', '-section', ColSecTag)
    ColSecTag = 4 # new section ID
    
    # Elastic section for interior part of beam elements 
    ElBeamSec = 5 # tag for elastic section of beam (required in lumped plasticity approach)
    Abeam = HCol*BCol          # cross-sectional area of beam
    Ibeam = 1/12*BCol*HCol**3  # gross moment of inertia of beam
    ops.section('Elastic', ElBeamSec, Ec, Abeam, Ibeam, Gc, kv)

    # Define column elements
    # ----------------------
    ColIntTag = 1 # Integration tag
    np = 5 # Number of integration points along length of element
    ops.beamIntegration('Lobatto', ColIntTag, ColSecTag, np) # Lobatto integration scheme

    # Create the columns using Beam-column elements
    eleType = 'forceBeamColumn'
    ops.element(eleType, 11, 1, 3, ColTransfTag, ColIntTag)
    ops.element(eleType, 12, 2, 4, ColTransfTag, ColIntTag)
    ops.element(eleType, 21, 3, 5, ColTransfTag, ColIntTag)
    ops.element(eleType, 22, 4, 6, ColTransfTag, ColIntTag)
    
    # Define beam elments 
    # -----------------------------
    # Lets follow lumped plasticity approach with fiber discretized sections at hinges
    # This is the good old Beam with Hinges element: Scott and Fenves et al. 2006
    BeamIntTag = 2 
    Lpl = 0.5 * HBeam # lets take the half of the section depth as plastic hinge length, alternatively, see formulay by Priestley et al. 2007
    ops.beamIntegration('HingeRadau', BeamIntTag, BeamSecTag, Lpl, BeamSecTag, Lpl, ElBeamSec) # HingeRadau integration scheme
    
    # Create the beam elements
    ops.element(eleType, 13, 3, 4, BeamTransfTag, BeamIntTag)
    ops.element(eleType, 23, 5, 6, BeamTransfTag, BeamIntTag)
    
    # Define gravity loads
    # --------------------
    P = 0.1*fc*HCol*BCol  # 10% of axial capacity of columns

    # Create a Plain load pattern with a Linear TimeSeries
    ops.timeSeries('Linear', 1)
    ops.pattern('Plain', 1, 1)

    # Create nodal loads at nodes
    ops.load(3, 0.0, -P, 0.0)
    ops.load(4, 0.0, -P, 0.0)
    ops.load(5, 0.0, -P, 0.0)
    ops.load(6, 0.0, -P, 0.0) 
    
    # Alternatively, apply distributed loading along the beam
#     w = (2*P)/LBeam
#     ops.eleLoad('-ele', 13, '-type', '-beamUniform', -w, 0)
#     ops.eleLoad('-ele', 23, '-type', '-beamUniform', -w, 0)
    # Define nodal mass in terms of axial load on columns
    mass = P/g

    ops.mass(3, mass, mass, 0.0)
    ops.mass(4, mass, mass, 0.0)
    ops.mass(5, mass, mass, 0.0)
    ops.mass(6, mass, mass, 0.0)

In [ ]:
# Display the Numerical Model for Elastic Frame
generate_inelastic_model() # generate a clean model
gravityForces, gravityDisps = do_gravity(pflag = 1) # perform gravity analysis

# plot model with tag lebels
opsplt.plot_model('nodes','elements')
opsv.plot_model()

# Perform modal analysis
nModes = 2 # desired number of modes to calculate 
lambdas = ops.eigen(nModes) # eigenvalue analysis for nEigen modes
ops.modalProperties("-print", "-file", "ModalReport.txt", "-unorm")

# Display specific mode shape with scale factor of 20 using the active model
opsplt.plot_modeshape(1, 20)
opsplt.plot_modeshape(2, 20)

## <font color=blue>4.2.1. Nonlinear Static Pushover Analysis (NSPA)</font>

In [ ]:
def do_nspa(dmax, ctrlNode, ctrlDOF, nSteps, IOflag=0):
    """
    Procedure to carry out a non-cylic pushover of a model
    Args:
        dmax:        Maximum displacement to run analysis
        ctrlNode:    Node to control with the displacement integrator
        ctrlDOF:     DOF the loading is applied
        nSteps:      Number of steps
        IOflag:      Option to print details on screen (default = 0)
                     1; print analysis info at each analysis step

    Returns:

    """

    ops.wipeAnalysis()
    ops.constraints('Transformation')
    ops.numberer('Plain')
    ops.system('BandGeneral')
    LoadFactor = [0]
    DispCtrlNode = [0]
    tol = 1e-12  # Set the tolerance to use during the analysis
    iterInit = 10  # Set the initial max number of iterations
    maxIter = 1000  # Set the max number of iterations to use with other integrators
    dU = dmax / (1.0 * nSteps)
    ops.test('NormDispIncr', tol, maxIter)  # lets start with energyincr as test
    ops.algorithm('KrylovNewton')
    ops.integrator('DisplacementControl', ctrlNode, ctrlDOF, dU)
    ops.analysis('Static')

    # Set the initial values to start the while loop
    ok = 0.0
    step = 1.0
    loadf = 1.0
    # This feature of disabling the possibility of having a negative loading has been included.
    while step <= nSteps and ok == 0 and loadf > 0:
        ok = ops.analyze(1)
        loadf = ops.getTime()
        step += 1.0
        LoadFactor.append(loadf)
        DispCtrlNode.append(ops.nodeDisp(ctrlNode, ctrlDOF))

    if ok != 0:
        print("Displacement Control Analysis is FAILED")
        print('-------------------------------------------------------------------------')

    else:
        print("Displacement Control Analysis is SUCCESSFUL")
        print('-------------------------------------------------------------------------')

    if loadf <= 0:
        print("Stopped because of Load factor below zero:", loadf)
        print('------------------------------------------------------------------------')

    return LoadFactor, DispCtrlNode     

In [ ]:
# Nonlinear Static Pushover Analysis of the inelastic frame
generate_inelastic_model() # generate a clean model
gravityForces, gravityDisps = do_gravity() # perform gravity analysis first

# create a load pattern for pushove analysis n
# Create the load pattern and apply loads
ops.timeSeries('Linear', 2)  # Define the timeSeries for the load pattern
ops.pattern('Plain', 2, 2)  # Define load pattern -- generalized

# apply incremental loads using triangular force distribution
lat_forces = np.array([0.5,1])
lat_forces = lat_forces/np.sum(lat_forces) # # normalize the laterial forces so that LoadFactor = BaseShear
ops.load(3, lat_forces[0],0,0)
ops.load(5, lat_forces[1],0,0)

# perform the pushover analysis
BuildingHeight = 2*144.0*inch
LoadFactor, DispCtrlNode = do_nspa(dmax=0.1*BuildingHeight, ctrlNode=5, ctrlDOF=1, nSteps=5000, IOflag=0)
plt.figure()
plt.plot(DispCtrlNode,LoadFactor)
plt.xlabel('Top displacement [m]')
plt.ylabel('Base Shear Foce [kN]')
plt.grid(True)

## <font color=blue>4.2.2. Nonlinear Response History Analysis (NRHA)</font>

In [ ]:
def do_nrha(tNode, bNode, Dt, Tmax, Dc):
    """
    Function to perform Non-linear Response History Analysis
    Args:
        tNode:    top nodes for the drift calculation
        bNode:    bottom nodes for the drift calculation
        Dt:       Analysis time step
        Tmax:     Length of the record (including padding of 0's)
        Dc:       Drift capacity for pier drift (%)

    Returns:
        mdrft:    Peak Interstorey Drifts [%]
    """
    # Set some analysis settings
    tol = 1.0e-7  # Set the Tolerance, so it can be referred back to (default)
    maxIter = 50  # Set the Max Number of Iterations (default)
    # Parameters required in Newmark Integrator
    # gamma = 1/2, beta = 1/4 --> Average Acceleration Method; Unconditionally stable
    # gamma = 1/2, beta = 1/6 --> Linear Acceleration Method; Unconditionally stable if Dt / T > 0.551
    gamma = 0.5
    beta = 0.25 
    ops.wipeAnalysis()
    ops.constraints('Transformation')
    ops.numberer('Plain')
    ops.system('BandGeneral')
    ops.integrator('Newmark', gamma, beta)
    ops.test('NormDispIncr', tol, maxIter)
    ops.algorithm('KrylovNewton')
    ops.analysis('Transient')
    
    # Set up analysis parameters
    cIndex = 0  # Initially define the control index (-1 for non-converged, 0 for stable, 1 for global collapse)
    controlTime = 0.0  # Start the controlTime
    ok = 0  # Set the convergence to 0 (initially converged)
    mflr = 0  # Set the initial pier collapse location
    Mdrft = 0.0  # Set initially the maximum of all storey drifts

    # Set up the storey drift values
    h = []
    mdrft = []
    
    
    for i in range(len(tNode)):
        # Find the coordinates of the nodes in Global Y (2)
        top2 = ops.nodeCoord(tNode[i], 2)
        bot2 = ops.nodeCoord(bNode[i], 2)
        dist = top2 - bot2
        h.append(dist)  # Current pier height
        mdrft.append(0.0)  # We will populate the lists with zeros initially
        if dist == 0: print("WARNING: Zerolength found in drift check")
            
    # Run the actual analysis now
    while cIndex == 0 and controlTime <= Tmax and ok == 0:
        # Set the default analysis parameters
        # Do the analysis
        ok = ops.analyze(1, Dt)  # Run a step of the analysis
        controlTime = ops.getTime()  # Update the control time

        # Bye bye...  Failed to converge, exit the analysis.
        if ok != 0:
            print(" ~~~ Failed at %.2f - exit the analysis......" % controlTime)
            ops.wipe()
            cIndex = -1

        if ok == 0:
            # Save here whatever you want, lets record peak interstorey drift ratios as an example (mdrft).
            
            # Check the storey drifts
            for i in range(len(tNode)):
                tNode_disp = ops.nodeDisp(tNode[i], 1)  # Current top node disp in dof 1
                bNode_disp = ops.nodeDisp(bNode[i], 1)  # Current bottom node disp in dof 1
                cHt = h[i]  # Current column height
                cdrft = 100.0 * abs(tNode_disp - bNode_disp) / cHt  # Current storey drift in dof 1 [%]
                if cdrft >= mdrft[i]: mdrft[i] = cdrft
                if cdrft > Mdrft: Mdrft = cdrft; mflr = i + 1  # Update the current maximum storey drift and where it is

            if Mdrft >= Dc: 
                cIndex = 1 
                Mdrft = Dc
                ops.wipe()  # Set the state of the model to local collapse (=1)

    if cIndex == -1:
        Analysis = "Analysis is FAILED to converge at %.3f of %.3f" % (controlTime, Tmax)
    if cIndex == 0:
        text = ["\nInterstorey drift: %.2f%% at floor %d" % (mdrft[i],i+1) for i in range(len(mdrft))]
        Analysis = ''.join(['Analysis is SUCCESSFULLY completed']+text)
    if cIndex == 1:
        Analysis = "Analysis is STOPPED, peak interstorey drift ratio, %d%%, is exceeded, global COLLAPSE is observed" % Dc

    print(Analysis)
    
    return mdrft

In [ ]:
# Nonlinear Response History Analysis of the inelastic frame
generate_inelastic_model() # generate a clean model
gravityForces, gravityDisps = do_gravity() # perform gravity analysis first

record_filename = 'tools//test_motion.txt' # acceleration time history
dt = 0.01*sec # time step of record
A_g = np.loadtxt(record_filename) # load the file
SF = 2.0 # scale factor applied on the record
gmDir = 1
xi = 0.05

# Set rayleigh damping based on first two modes
Lambda = np.asarray(ops.eigen(2))
Omega = Lambda ** 0.5
wi = Omega[0]
wj = Omega[1]
alpha = 2.0 * xi * wi * wj / (wi + wj)
beta = 2.0 * xi / (wi + wj)
ops.rayleigh(alpha, 0.0, 0.0, beta)

# Define the load pattern
load_tag_dynamic = 2 # tag for time series to use
pattern_tag_dynamic = 2 # tag for load pattern to use
values = list(-1 * A_g)  # should be negative
ops.timeSeries('Path', load_tag_dynamic, '-dt', dt, '-values', *values, '-factor', SF) # time series object
ops.pattern('UniformExcitation', pattern_tag_dynamic, gmDir, '-accel', load_tag_dynamic) # pattern object

tNode = [3, 5] # these are top floor nodes
bNode = [1, 3] # these are bottom floor nodes
Dc = 10 # global collapse limit we stop the analysis if this treshold is exceeded
Tmax = len(A_g)*dt + 10 # total analysis time, you may add a free vibration
mdrft = do_nrha(tNode, bNode, dt, Tmax, Dc)